## Step 1 Scraping

In [23]:
from urllib.parse import urlsplit
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from splinter import Browser
from urllib.request import urlopen, urlretrieve
from urllib.parse import urljoin
import os
import urllib
import pandas as pd
import time
def init_browser():
#pointing to the directory where chromedriver exists
    exec_path = {'executable_path': '/Users/ghassan/Downloads/chromedriver'}

    return Browser('chrome', headless=False, **exec_path)

## viewing and scraping Mars related Web Sites

In [25]:
def scrape():
#visiting web pages
    url1 = 'https://mars.nasa.gov/news/'
    url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    url3 = 'https://twitter.com/marswxreport?lang=en'
    url4 = 'http://space-facts.com/mars/'
    url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

    db_mars_collection ={}
#BEG  NASA
    browser = init_browser()
# visit NASA web for two text items
    browser.visit(url1)
#using bs to write it into html
    html = browser.html
# making soup
    soup = bs(html,"html.parser")
#mandaitory  sleep for two seconds
    time.sleep(2)
#we have the soup let find the beanas
    news_title = soup.find("div",class_="content_title").text
    news_p = soup.find("div", class_="article_teaser_body").text
    print(f"Title: {news_title}")
    print(f"Paragraph: {news_p}")

#building the dictionary
    db_mars_collection['news_title'] = news_title
    db_mars_collection['news_parapgraph'] = news_p
#END  NASA

#BEG JPL
#initialize browser for second web site
    browser=init_browser()
#visiting the JPL page url2
    browser.visit(url2)
    basename_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url2))
    print(basename_url)
    time.sleep(2)


#click on FULL IMAGE and use by_css to find image URL    
    browser.click_link_by_partial_text('FULL IMAGE')
    featured_image_url = browser.find_by_css('.fancybox-image').first['src']

    db_mars_collection['featured_image_url'] = featured_image_url



#END  JPL
#BEG Mars Weather  twitter
    browser = init_browser()
    browser.visit(url3)
    for text in browser.find_by_css('.tweet-text'):
        if text.text.partition(' ')[0] == 'Sol':
            mars_weather = text.text
            break
    db_mars_collection['mars_weather'] = mars_weather
#EMD Mars Weather  twitter

#BEG  FACTS TABLE 
    time.sleep(2)
    browser=init_browser()
    browser.visit(url4)
#visiting the facts  page url4
    table = pd.read_html(url4)
    df = table[0]
    df.columns = ["Parameter", "Values"]
    df = df.set_index(["Parameter"])
    df = df.replace("\n", "")
    mars_facts = df.to_html(justify='left')
    db_mars_collection['mars_facts'] = mars_facts
#END  FACTS TABLE 
## No soup just splinter 
    browser = init_browser()
    browser.visit(url5)
#Find titles with tag h3 and assign     
    title1 = browser.find_by_tag('h3')[0].text
    title2 = browser.find_by_tag('h3')[1].text
    title3 = browser.find_by_tag('h3')[2].text
    title4 = browser.find_by_tag('h3')[3].text
    
#click on the class='thumb', next page text "Sample" take you to image location    
    browser.find_by_css('.thumb')[0].click()
    first_img = browser.find_by_text('Sample')['href']
    browser.back()

    browser.find_by_css('.thumb')[1].click()
    second_img = browser.find_by_text('Sample')['href']
    browser.back()

    browser.find_by_css('.thumb')[2].click()
    third_img = browser.find_by_text('Sample')['href']
    browser.back()

    browser.find_by_css('.thumb')[3].click()
    fourth_img = browser.find_by_text('Sample')['href']

    hemisphere_image_urls = [
        {'title': title1, 'img_url': first_img},
        {'title': title2, 'img_url': second_img},
        {'title': title3, 'img_url': third_img},
        {'title': title4, 'img_url': fourth_img}
    ]
    db_mars_collection['hemisphere_image_urls'] = hemisphere_image_urls
    return(db_mars_collection)
    

In [26]:
scrape()

Title: MarCO Makes Space for Small Explorers
Paragraph: A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.
https://www.jpl.nasa.gov/


{'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18358_ip.jpg',
 'hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: left;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Parameter</th>\n      <th></th>\n    </tr>\n  </thead>\n 